In [12]:
# -*- coding: utf-8 -*-
# ResNet-18 train -> infer -> submission.csv

import os, re, json, time, argparse
from pathlib import Path
from typing import List
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


import glob
import matplotlib.pyplot as plt
import random
from tqdm.auto import tqdm
import cv2
import numpy as np
# --------------------- config ---------------------
TRAIN_DIR = Path("trainset")
TEST_DIR  = Path("testset")
OUT_DIR   = Path("./outputs"); OUT_DIR.mkdir(exist_ok=True, parents=True)

import os
import glob
os.chdir('/Users/youyu-ling/Downloads/AI02/')
print(os.listdir('.'))
import os

defect_classes = sorted([d for d in os.listdir('trainset')])
print(f"📌 缺陷類別數量: {len(defect_classes)}")
print(f"📌 缺陷類別名稱: {defect_classes}")
train_data_dir = 'trainset'
test_data_dir = 'testset'

train_image_paths = []
test_image_paths = []

# Load training images from subdirectories (assuming .jpg files)
for defect_class in defect_classes:
    train_class_dir = os.path.join(train_data_dir, defect_class)
    if os.path.exists(train_class_dir):
        train_image_paths.extend(glob.glob(os.path.join(train_class_dir, '*.jpg')))
        print(f"在 {train_class_dir} 中找到 {len(glob.glob(os.path.join(train_class_dir, '*.jpg')))} 張 .jpg 圖片")


# Load testing images directly from the testset directory (assuming .jpg files)
if os.path.exists(test_data_dir):
    test_image_paths.extend(glob.glob(os.path.join(test_data_dir, '*.jpg')))
    print(f"在 {test_data_dir} 中找到 {len(glob.glob(os.path.join(test_data_dir, '*.jpg')))} 張 .jpg 圖片")


print(f"📌 訓練圖片總數: {len(train_image_paths)}")
print(f"📌 測試圖片總數: {len(test_image_paths)}")
print(f"📌 圖片總數: {len(train_image_paths) + len(test_image_paths)}")

# The test set distribution cannot be determined directly from the file paths
# as they are not in class-specific folders.
# We will address labeling the test set later.
print("📌 測試集分佈：無法直接從檔案路徑判斷，因為檔案未依類別分類。")


# Define image size for preprocessing
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Function to load and preprocess images
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img.astype('float32') / 255.0  # Normalize pixel values
    return img

# Prepare training data and labels
train_images = []
train_labels = []

for defect_class in defect_classes:
    train_class_dir = os.path.join(train_data_dir, defect_class)
    class_index = defect_classes.index(defect_class) # Get numerical label for the class
    for image_path in glob.glob(os.path.join(train_class_dir, '*.jpg')):
        preprocessed_img = preprocess_image(image_path)
        if preprocessed_img is not None:
            train_images.append(preprocessed_img)
            train_labels.append(class_index)

# Prepare testing data (no labels needed for now, as they are for prediction)
test_images = []
# Assuming test images are directly in test_data_dir and are .jpg
for image_path in glob.glob(os.path.join(test_data_dir, '*.jpg')):
     preprocessed_img = preprocess_image(image_path)
     if preprocessed_img is not None:
         test_images.append(preprocessed_img)


# Convert lists to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)

print(f"訓練圖片資料形狀: {train_images.shape}")
print(f"訓練標籤資料形狀: {train_labels.shape}")
print(f"測試圖片資料形狀: {test_images.shape}")

['trainset', 'hw2_local.py', 'testset', 'sample_code_2.ipynb', 'ai01.py', 'outputs']
📌 缺陷類別數量: 8
📌 缺陷類別名稱: ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Near-full', 'Random', 'Scratch']
在 trainset/Center 中找到 3462 張 .jpg 圖片
在 trainset/Donut 中找到 409 張 .jpg 圖片
在 trainset/Edge-Loc 中找到 2417 張 .jpg 圖片
在 trainset/Edge-Ring 中找到 8554 張 .jpg 圖片
在 trainset/Loc 中找到 1620 張 .jpg 圖片
在 trainset/Near-full 中找到 54 張 .jpg 圖片
在 trainset/Random 中找到 609 張 .jpg 圖片
在 trainset/Scratch 中找到 500 張 .jpg 圖片
在 testset 中找到 7894 張 .jpg 圖片
📌 訓練圖片總數: 17625
📌 測試圖片總數: 7894
📌 圖片總數: 25519
📌 測試集分佈：無法直接從檔案路徑判斷，因為檔案未依類別分類。
訓練圖片資料形狀: (17625, 128, 128, 3)
訓練標籤資料形狀: (17625,)
測試圖片資料形狀: (7894, 128, 128, 3)


In [22]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. 參數定義 (根據你的數據輸出) ---
# 1. 類別定義
defect_classes = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Near-full', 'Random', 'Scratch']
num_classes = len(defect_classes)

# 2. 圖像尺寸 (根據你的預處理輸出)
IMG_HEIGHT = 128
IMG_WIDTH = 128
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# 3. 訓練參數
BATCH_SIZE = 32
EPOCHS = 10  # 初始階段，可以先跑幾個 Epoch 看看效果
LEARNING_RATE = 0.001

In [23]:
# --- 2. 數據準備與分割 (請在這裡放入你的 NumPy 陣列變量) ---
# 假設你的變量已經計算出來: train_images, train_labels, test_images

# 數據分割：將訓練集分成訓練集和驗證集 (80% 訓練, 20% 驗證)
X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

# 創建 tf.data.Dataset 對象
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(BATCH_SIZE)
validation_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_ds_for_predict = tf.data.Dataset.from_tensor_slices(test_images).batch(BATCH_SIZE)

print(f"訓練集大小: {len(X_train)} | 驗證集大小: {len(X_val)} | 測試集大小: {len(test_images)}")

訓練集大小: 14100 | 驗證集大小: 3525 | 測試集大小: 7894


In [27]:
# --- 3. 模型建構 (使用你最初的 Keras 代碼) ---

# 載入基礎模型
base_model = MobileNetV2(weights='imagenet', 
                         include_top=False, 
                         input_shape=INPUT_SHAPE)

# Freeze the layers of the base model so they are not trained
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x) # Add a global spatial average pooling layer
x = Dense(128, activation='relu')(x) # Add a dense layer
predictions = Dense(num_classes, activation='softmax')(x) # Add the final classification layer

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,422,984 (9.24 MB)

 Trainable params: 165,000 (644.53 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [28]:
# --- 4. 模型訓練 ---
print("\n--- 開始訓練 (階段 1: 僅訓練新加的頂層) ---")

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS
)

print("--- 訓練完成 ---")


--- 開始訓練 (階段 1: 僅訓練新加的頂層) ---
Epoch 1/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 60s 123ms/step - accuracy: 0.8636 - loss: 0.3788 - val_accuracy: 0.8919 - val_loss: 0.2932
Epoch 2/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 52s 118ms/step - accuracy: 0.9107 - loss: 0.2365 - val_accuracy: 0.9038 - val_loss: 0.2496
Epoch 3/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 52s 118ms/step - accuracy: 0.9248 - loss: 0.2011 - val_accuracy: 0.8874 - val_loss: 0.2883
Epoch 4/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 52s 119ms/step - accuracy: 0.9298 - loss: 0.1867 - val_accuracy: 0.8993 - val_loss: 0.2669
Epoch 5/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 53s 121ms/step - accuracy: 0.9357 - loss: 0.1746 - val_accuracy: 0.9240 - val_loss: 0.2237
Epoch 6/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 55s 124ms/step - accuracy: 0.9426 - loss: 0.1522 - val_accuracy: 0.9146 - val_loss: 0.2442
Epoch 7/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 53s 121ms/step - accuracy: 0.9465 - loss: 0.1439 - val_accuracy: 0.9061 - val_loss: 0.2830
Epoch 8/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 56s 126ms/st

In [29]:
# --- 5. 模型評估與預測 ---

# 評估模型在驗證集上的表現
loss, accuracy = model.evaluate(validation_ds)
print(f"\n驗證集準確率 (Validation Accuracy): {accuracy*100:.2f}%")


# --- 進行測試集預測 ---
print("\n--- 開始進行測試集預測 ---")
test_predictions_prob = model.predict(test_ds_for_predict)
test_predictions_labels = np.argmax(test_predictions_prob, axis=1)

# 將數值標籤轉換回類別名稱
predicted_defect_names = [defect_classes[label] for label in test_predictions_labels]

# 最終你可以在這裡創建 submission.csv
# 例如，如果你知道測試圖片的原始文件名順序：
# import pandas as pd
# test_filenames = [f.name for f in Path(test_data_dir).glob('*.jpg')]
# submission_df = pd.DataFrame({
#     'filename': test_filenames,
#     'predicted_defect': predicted_defect_names
# })
# submission_df.to_csv("submission.csv", index=False)
# print("Submission file 'submission.csv' created.")

111/111 ━━━━━━━━━━━━━━━━━━━━ 11s 99ms/step - accuracy: 0.9211 - loss: 0.2448

驗證集準確率 (Validation Accuracy): 92.11%

--- 開始進行測試集預測 ---
247/247 ━━━━━━━━━━━━━━━━━━━━ 28s 105ms/step


In [34]:
import pandas as pd       
predicted_defect_names = [defect_classes[label] for label in test_predictions_labels]

# 測試集 ID 是從 0 到 7893
submission_df = pd.DataFrame({
    'ID': np.arange(len(test_filenames)), # 使用索引作為ID
    'Category': predicted_defect_names
})

# 儲存為 CSV 檔案
output_filename = "submission.csv"
submission_df.to_csv(output_filename, index=False)

print(f"\n--- 最終結果 ---")
print(f"✅ 成功生成預測結果檔案: {output_filename}")
print(f"檔案包含 {len(submission_df)} 筆預測結果。")
print("--- 檔案頭部預覽 ---")
print(submission_df.head())

NameError: name 'test_filenames' is not defined